# Hands-on Lab: Acquiring and Processing Information on the World's Largest Banks

# Project Scenario:
You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

# Project tasks
Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

Task 6:
Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [10]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime 
import numpy as np

url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
output_path = './Largest_banks_data.csv'


def log_progress(message):
    
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S'       # Year-Monthname-Day-Hour-Minute-Second 
    
    now = datetime.now()                         # get current timestamp 
    
    timestamp = now.strftime(timestamp_format) 
    
    with open("./code_log.txt", "a") as f: 
        
        f.write(timestamp + ' : ' + message + '\n')
    

def extract(url, table_attribs):
    
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    page = requests.get(url).text                  
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns = table_attribs)    
    tables = data.find_all('tbody')             
                                                                                 
    rows = tables[0].find_all('tr')    
                                                                             
    for row in rows:
    
        if row.find("td") is not None:
        
            col = row.find_all('td')
        
            data_dict = {"Name": col[1].find_all("a")[1]["title"], "MC_USD_Billion": float(col[2].contents[0][:-1])}
                        
            df1 = pd.DataFrame(data_dict, index = [0])
                
            df = pd.concat([df,df1], ignore_index = True)
                                                    
    return df
 

def transform(df, csv_path):
    
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    
    # Load the exchange rates from the CSV file and convert it to a dictionary
    exchange_rate = pd.read_csv(csv_path).set_index('Currency').to_dict()['Rate']
    
    # Add the columns by scaling the 'MC_USD_Billion' column using the exchange rates
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]
    
    return df


def load_to_csv(df, output_path):
    
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    
    df.to_csv(output_path)

def load_to_db(df, sql_connection, table_name):
    
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    
    df.to_sql(table_name, sql_connection, if_exists = 'replace', index = False)

def run_query(query_statement, sql_connection):
    
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
    
pd.set_option('display.max_colwidth', None)

# Adjust the number of rows and columns to display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Adjust the display width
pd.set_option('display.width', 1000)

log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, csv_path)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, output_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Executing the queries')

query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT Name FROM {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()


SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion
0                           JPMorgan Chase          432.92          346.34          402.62        35910.71
1                          Bank of America          231.52          185.22          215.31        19204.58
2  Industrial and Commercial Bank of China          194.56          155.65          180.94        16138.75
3               Agricultural Bank of China          160.68          128.54          149.43        13328.41
4                                HDFC Bank          157.91          126.33          146.86        13098.63
5                              Wells Fargo          155.87          124.70          144.96        12929.42
6                                     HSBC          148.90          119.12          138.48        12351.26
7                           Morgan Stanley          140.83          112.66          130.97        11681.85
8        